In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import missingno as mno
import plotly.offline as pyo 
import plotly.figure_factory as ff
import plotly.io as pio
from wordcloud import WordCloud
color_pal = sns.color_palette()
plt.style.use('seaborn-dark-palette')
plt.style.use('dark_background')

import nltk
import re

import warnings
warnings.filterwarnings('ignore')
sns.set_theme(style='darkgrid', palette='colorblind')
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()

#Model
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
df=pd.read_csv("/kaggle/input/employee-salaries-for-different-job-roles/ds_salaries.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.describe().T

In [ ]:
df.describe(include='object').T

In [ ]:
df.shape

In [ ]:
unique_values = df.nunique()
unique_values

In [ ]:
print(df.isna().sum())
df.isna().sum().plot(kind='bar')
plt.show()
sns.barplot(df.isna())
plt.xticks(rotation=45)
plt.show()

In [ ]:
df.duplicated().sum()

In [ ]:
# Finding duplicate rows
duplicate_rows = df[df.duplicated(keep='first')]

# Number of duplicate rows
num_duplicates = duplicate_rows.shape[0]

# Displaying the duplicate rows
print(f"Number of duplicate rows: {num_duplicates}")
duplicate_rows

In [ ]:
# Import the ProfileReport class from the ydata_profiling library
from ydata_profiling import ProfileReport

# Create a comprehensive profile report for the DataFrame 'df'
# This report will contain various statistics, insights, and visualizations about the data
profile = ProfileReport(df)

In [ ]:
profile                     # or this one
profile.to_notebook_iframe() # use this line to show the output

In [ ]:
locationLocation_of_Studnet = df.groupby('company_size')['salary'].sum().reset_index()

# Create a line plot using Plotly Express
# x-axis: locationLocation, y-axis: Location ,
# labels for the x-axis, title, and customized height
locationLocation_of_Studnet = px.line(
    locationLocation_of_Studnet,  # DataFrame containing the data
    x='company_size',   # x-values
    y='salary',  # y-values
    title='salary  by company_size',  # Set the title of the plot
    height=800  # Set the height of the plot
)

# Display the plot
locationLocation_of_Studnet.show()


locationLocation_of_Studnet = df.groupby('job_title')['salary'].sum().reset_index()
# Create a line plot using Plotly Express
# x-axis: locationLocation, y-axis: Location ,
# labels for the x-axis, title, and customized height
locationLocation_of_Studnet = px.line(
    locationLocation_of_Studnet,  # DataFrame containing the data
    x='job_title',   # x-values
    y='salary',  # y-values
    title='salary  by job_title',  # Set the title of the plot
    height=800  # Set the height of the plot
)

# Display the plot
locationLocation_of_Studnet.show()




In [ ]:
# Correlation:
plt.figure(figsize=(10, 6))
sns.heatmap(df.drop(['Unnamed: 0'], axis = 1).corr(), cmap="Blues", annot = True)

In [ ]:
# Calculate the value counts for each unique value in the 'job_title' column
top10_Location = df['job_title'].value_counts()[:10]

# Create a bar plot using Plotly Express
# y-axis: Count of each job title, x-axis: Location,
# color-coded by job titles and using a color palette
fig = px.bar(
    y=top10_Location.values,  # y-values: count of each job title
    x=top10_Location.index,   # x-values: job titles
    color=top10_Location.index,  # Color the bars based on job titles
    color_discrete_sequence=px.colors.sequential.PuBuGn,  # Set color palette
    text=top10_Location.values,  # Display the count values on top of the bars
    title='Top 10 job_title',  # Set the title of the plot
    template='plotly_white'  # Use a dark template for the plot
)

# Update the layout of the plot
fig.update_layout(
    title_text='job_title Distribution',  # Set the title of the plot
    height=700,  # Set the height of the plot
    xaxis_title="job_title",  # Label for the x-axis
    yaxis_title="Count",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
df['company_size'].value_counts().plot(kind = 'barh', figsize = (10,2), grid = True, color='black')
plt.title('company_size Amount')
plt.xlabel('Amount')
plt.ylabel('company_size')
plt.show()

In [ ]:
# Calculate the average salary for each Hotel Name and sort in descending order
average_Price = df.groupby('experience_level')['salary'].mean().reset_index()
average_Price = average_Price.sort_values(by='salary', ascending=False)

# Select the top 10 Hotel Name with the highest average price
top_10_Price = average_Price.head(10)

# Create a new figure using Plotly
fig = go.Figure()

# Add a bar trace to the figure
fig.add_trace(go.Bar(
    x=top_10_Price['experience_level'],  # x-values: Hotel Name
    y=top_10_Price['salary']  # y-values: average Price
))

# Update the layout and appearance of the plot
fig.update_layout(
    title='Top 10 experience_level by salary',  # Set the title of the plot
    xaxis_title='experience_level',  # Label for the x-axis
    yaxis_title='salary',  # Label for the y-axis
    #paper_bgcolor='#595964',  # Background color of the plot
    template='plotly_white',  # Use a dark template for the plot
    font=dict(color='black'),  # Set font color to white
    height=800  # Set the height of the plot
)

# Initialize Plotly for notebook integration
pyo.init_notebook_mode(connected=True)

# Display the plot
pyo.iplot(fig)

In [ ]:
# Calculate the average salary for each Hotel Name and sort in descending order
average_Reviews = df.groupby('job_title')['salary'].mean().reset_index()
average_Reviews = average_Reviews.sort_values(by='salary', ascending=False)

# Select the top 10 Hotel Name with the highest average price
top_10_Reviews = average_Reviews.head(10)

# Create a new figure using Plotly
fig = go.Figure()

# Add a bar trace to the figure
fig.add_trace(go.Bar(
    x=top_10_Reviews['job_title'],  # x-values: Hotel Name
    y=top_10_Reviews['salary']  # y-values: Reviews Price
))

# Update the layout and appearance of the plot
fig.update_layout(
    title='Top 10 job_title by salary',  # Set the title of the plot
    xaxis_title='job_title',  # Label for the x-axis
    yaxis_title='salary',  # Label for the y-axis
    #paper_bgcolor='#595964',  # Background color of the plot
    template='plotly_white',  # Use a dark template for the plot
    font=dict(color='black'),  # Set font color to white
    height=900  # Set the height of the plot
)

# Initialize Plotly for notebook integration
pyo.init_notebook_mode(connected=True)

# Display the plot
pyo.iplot(fig)

In [ ]:
# Calculate the value counts for each unique value in the 'employment_type' column
type_grouped = df['employment_type'].value_counts()

# Define the desired order of employment types
e_type = ['Full-Time', 'Part-Time', 'Contract', 'Freelance']

# Create a bar plot using Plotly Express
# x-axis: Employment types, y-axis: Count of each employment type,
# color-coded by employment type, and use a predefined color palette
fig = px.bar(
    x=e_type,  # x-values: employment types
    y=type_grouped.values,  # y-values: counts of each employment type
    color=type_grouped.index,  # Color the bars based on employment type
    color_discrete_sequence=px.colors.sequential.PuBuGn,  # Set color palette
    template='plotly_white',  # Use a dark template for the plot
    text=type_grouped.values  # Display the count values on top of the bars
)

# Update the layout and appearance of the plot
fig.update_layout(
    title_text='Employment Type Distribution',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="Employment Type",  # Label for the x-axis
    yaxis_title="Count",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic"),  # Set the font size and family for the text
)

# Adjust the width of the bars
fig.update_traces(width=0.5)

# Display the plot
fig.show()

In [ ]:
# Replace 'EN' with 'Entry-level/Junior' in the 'experience_level' column
df['experience_level'] = df['experience_level'].replace('EN', 'Entry-level/Junior')

# Replace 'MI' with 'Mid-level/Intermediate' in the 'experience_level' column
df['experience_level'] = df['experience_level'].replace('MI', 'Mid-level/Intermediate')

# Replace 'SE' with 'Senior-level/Expert' in the 'experience_level' column
df['experience_level'] = df['experience_level'].replace('SE', 'Senior-level/Expert')

# Replace 'EX' with 'Executive-level/Director' in the 'experience_level' column
df['experience_level'] = df['experience_level'].replace('EX', 'Executive-level/Director')

# Calculate the value counts for each unique value in the 'experience_level' column
ex_level = df['experience_level'].value_counts()

# Create a bar plot using Plotly Express
# x-axis: Count of each experience level, y-axis: Experience level categories,
# color-coded by "experience_level"
fig = px.bar(ex_level, x=ex_level, y=ex_level.values, color="experience_level")

# Update the layout of the plot
fig.update_layout(
    title_text='Experience Level Distribution',  # Set the title of the plot
    height=500,  # Set the height of the plot
    xaxis_title="Count",  # Label for the x-axis
    yaxis_title="Experience Level",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
 # x-axis: "locationLocation", y-axis: "Number of student per staffs", color-coded by "Number of Studnet"
fig = px.bar(df, x="experience_level", y="salary", color="salary")

# Update the layout of the plot
fig.update_layout(
    title_text='experience_level Distribution with salary',  # Set the title of the plot
    height=650,  # Set the height of the plot
    xaxis_title="experience_level",  # Label for the x-axis
    yaxis_title="salary",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# x-axis: "work_year", y-axis: "salary_in_usd", color-coded by "experience_level",
# and create a notched box plot
fig = px.box(df, x="work_year", y="salary_in_usd", color="experience_level", notched=True)

# Update the layout of the plot
fig.update_layout(
    title_text='Salary Box Distribution',  # Set the title of the plot
    height=500,  # Set the height of the plot
    xaxis_title="Work Year",  # Label for the x-axis
    yaxis_title="Salary",  # Label for the y-axis
    font=dict(size=17, family="Franklin Gothic")  # Set the font size and family for the text
)

# Display the plot
fig.show()

In [ ]:
# Sunburst chart for the distribution of Rating_Description
fig2 = px.sunburst(df, path=['experience_level'], color_discrete_sequence=px.colors.qualitative.Set2)
fig2.update_layout(title_text='Distribution of experience_level', height=500)
fig2.show()

# Sunburst chart for the distribution of Rating
fig2 = px.sunburst(df, path=['employment_type'], color_discrete_sequence=px.colors.qualitative.Set1)
fig2.update_layout(title_text='Distribution of employment_type', height=500)
fig2.show()

# Sunburst chart for the distribution of Rating_Description
fig2 = px.sunburst(df, path=['salary_currency'], color_discrete_sequence=px.colors.qualitative.Set2)
fig2.update_layout(title_text='Distribution of salary_currency', height=500)
fig2.show()

# Sunburst chart for the distribution of Rating
fig2 = px.sunburst(df, path=['company_size'], color_discrete_sequence=px.colors.qualitative.Set1)
fig2.update_layout(title_text='Distribution of company_size', height=500)
fig2.show()


In [ ]:
fig = px.pie(df, values="salary", names="company_size",
             color_discrete_sequence=px.colors.sequential.RdBu,
             opacity=0.7, hole=0.5)
fig.show()

In [ ]:
import plotly.graph_objects as px
import pandas as pd
 
# reading the database
 
plot = px.Figure(data=[px.Scatter(
    x=df['experience_level'],
    y=df['salary'],
    mode='markers',)
])
 
# Add dropdown
plot.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["type", "scatter"],
                    label="Scatter Plot",
                    method="restyle"
                ),
                dict(
                    args=["type", "bar"],
                    label="Bar Chart",
                    method="restyle"
                )
            ]),
            direction="down",
        ),
    ]
)
 
plot.show()

In [ ]:
import plotly.express as px
fig = px.scatter(df, x="experience_level", y="salary", color="employment_type")
fig.show()

fig = px.scatter(df, x="salary", y="employment_type", size="salary",color="experience_level",
           hover_name="company_location", log_x=True, size_max=60)
fig.show()

In [ ]:
df.head()

In [ ]:
fig = px.funnel(df, x='salary', y='employment_type')
fig.show()

In [ ]:
fig = px.scatter_matrix(df, height=1800, width=2000)
fig.show()

In [ ]:
fig = px.box(df, x="experience_level", y="salary")
 
# showing the plot
fig.show()

In [ ]:
# Box Plot
# Prepare data for the box plot
hist_data = [df['salary']]
group_labels = ['salary']

# Create the box plot using Plotly Express
fig1 = px.box(y=df['salary'], template='plotly_white', title='salary (BoxPlot)')

# Customize the layout of the box plot
fig1.update_layout(font=dict(size=17, family="Franklin Gothic"))

# Display the box plot
fig1.show()

# Dist Plot
# Prepare data for the distribution plot
hist_data = [df['salary']]
group_labels = ['salary']

# Create the distribution plot using Plotly Figure Factory
fig2 = ff.create_distplot(hist_data, group_labels, show_hist=False)
fig2.layout.template = 'plotly_white'

# Customize the layout of the distribution plot
fig2.update_layout(title='Prsalaryice (DistPlot)', font=dict(size=17, family="Franklin Gothic"))

# Display the distribution plot
fig2.show()

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Assuming you have already loaded the data into the 'df' DataFrame

# Extract subsets of the data based on work year
work_years = [2020, 2021, 2022]
colors = ['Aqua', 'pink', 'Gray']

# Prepare data for the histogram
hist_data = [df[df['work_year'] == year]['salary_in_usd'] for year in work_years]
group_labels = [f'{year} salary' for year in work_years]

# Calculate mean salaries for each work year
year_salary = df.groupby('work_year')['salary_in_usd'].mean()

# Create a Plotly figure for the bar chart
fig1 = go.Figure()

for i, year in enumerate(work_years):
    fig1.add_trace(go.Bar(
        x=[year],
        y=[year_salary[year]],
        name=group_labels[i],
        marker_color=colors[i],
        text=[f'{year_salary[year] / 1000:.2f}k'],
        textposition='auto'
    ))

# Customize the appearance of the Plotly figure
fig1.update_traces(width=0.3)
fig1.update_layout(
    xaxis_title="Work Year",
    yaxis_title="Mean Salary (k)",
    title='Mean Salary by Work Year',
    font=dict(size=17, family="Franklin Gothic"),
    template='plotly_white',
    height=500
)

# Display the Plotly figure
fig1.show()

# Extract subsets of the data based on experience level
experience_levels = ['Entry-level/Junior', 'Mid-level/Intermediate', 'Senior-level/Expert', 'Executive-level/Director']
colors = ['Yellow','Aqua', 'pink', 'Gray']

# Prepare data for the histogram
hist_data = [df[df['experience_level'] == level]['salary_in_usd'] for level in experience_levels]

# Calculate mean salaries for each experience level group
mean_salaries = [data.mean() for data in hist_data]

# Create a Plotly figure for the bar chart
fig2 = go.Figure()

for i, level in enumerate(experience_levels):
    fig2.add_trace(go.Bar(
        x=[level],
        y=[mean_salaries[i]],
        name=level,
        marker_color=colors[i],
        text=[f'{mean_salaries[i] / 1000:.2f}k'],
        textposition='auto'
    ))

# Customize the appearance of the Plotly figure
fig2.update_traces(width=0.4)
fig2.update_layout(
    xaxis_title="Experience Level",
    yaxis_title="Mean Salary (k)",
    title='Mean Salary by Experience Level',
    font=dict(size=17, family="Franklin Gothic"),
    template='plotly_white',
    height=500
)

# Display the Plotly figure
fig2.show()


In [ ]:
df.head()

In [ ]:
# Select columns with object (categorical) data types
num_cols = df.select_dtypes(include='object').columns.tolist()

# Initialize the LabelEncoder
le = LabelEncoder()

# Apply Label Encoding to the selected numerical columns
for x in num_cols:  
    df[x] = le.fit_transform(df[x])

# Now, your categorical columns (excluding column 0) have been converted to numerical values
df.head()

In [ ]:
Corr_Matrix = df.corr()

# Set up the figure and plot the heatmap
plt.figure(figsize=(30, 30))
sns.heatmap(Corr_Matrix, annot=True, cmap='RdYlGn', center=0)
plt.show()

In [ ]:
print('Top 5 Most Positively Correlated to the To salary_in_usd')
Corr_Matrix['company_location'].sort_values(ascending=False).head(5)

In [ ]:
print('Top 5 Most Positively Correlated to salary_in_usd ')
Corr_Matrix['salary_in_usd'].sort_values(ascending=True).head(5)

In [ ]:
df.head()

In [ ]:
X = df.drop(columns=['salary_in_usd','Unnamed: 0'])
y = df['salary_in_usd']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
models = {
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
}
best_model = None
best_r2 = 0

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred= model.predict(X_test)

    # Evaluate the model
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    submit = pd.DataFrame()
    submit['Actual_salary'] = y_test
    submit['Predict_salary'] = y_pred
    submit = submit.reset_index()
    r2 = r2_score(y_test, y_pred)
    if r2 > best_r2:
        best_r2 = r2
        best_model = model.__class__.__name__

    print(f'{model_name}:')
    print(f'R2 Score: {r2:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')
    print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
    print(submit.head(5))

    print('----------------------------------------')
print(f"The best performing model is: {best_model} with accuracy: {best_r2:.2f}")